In [55]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [2]:
prefix = 'https://gfc-russia.ru'
webpage_response = requests.get('https://gfc-russia.ru/catalog.html')

webpage = webpage_response.content
soup = BeautifulSoup(webpage, 'html.parser')

In [74]:
catalog_links = soup.find_all('a', attrs={'class': 'a88'})

links = []
strings = []
for link in catalog_links:
    links.append(prefix+link['href'])
    strings.append(link.span.string)

In [75]:
d = {'category': np.array(strings), 'link': np.array(links)}
links_df = pd.DataFrame(d)
links_df = links_df.loc[~links_df['category'].isin([None])]

In [144]:
products_df = pd.DataFrame({'name' : [], 'price' : [], 'product_id' : [], 'category': [], 'product_page': []})

Empty DataFrame
Columns: [name, price, product_id, category, product_page]
Index: []


In [145]:
for index, row in links_df.iterrows():
    webpage = requests.get(row['link'])
    soup = BeautifulSoup(webpage.content, 'html.parser')
    products = soup.find_all('div', attrs={'class': 'product_item'})
    for prod in products:
        name = prod['data-product_name']
        prod_id = int(prod['data-product_id'])
        prod_price = prod.find('div', attrs={'data-price_type': 'РОЗНИЦА Москва'})
        price = float(prod_price['data-price_value'])
        prod_page = prod.find('a', attrs={'class': 'product_img'})
        page_link = prefix + prod_page['href']
        products_df = products_df.append({'name' : name, 'price' : price, 'product_id' : prod_id, 'category' : row['category'], 'product_page': page_link}, ignore_index=True)
print(products_df)

                                                  name   price  product_id  \
0    Угорь жареный мороженный 50% соуса 9/10/11  oz...  1275.5     11992.0   
1         Угорь жареный замороженный технический 11 OZ  1836.5     12106.0   
2    Филе желтоперого тунца Loin с/м  2-5кг, 1кг Re...   969.0      6950.0   
3      Филе желтоперого тунца САКУ с/м  500-800гр, 1кг  1173.5     12135.0   
4    Филе желтоперого тунца САКУ с/м  300-500гр, 1к...  1479.0       756.0   
..                                                 ...     ...         ...   
620                     Топпинг Малина 1л  Дольче Роса   280.0      1544.0   
621                       Сироп Клубника 1 л Спум Фудс   290.0      1551.0   
622                       Сироп Бабл-гам 1 л Спум Фудс   290.0      1966.0   
623                         Сироп Малина 1 л Спум Фудс   290.0      1967.0   
624  Бамбуковая пароварка двойная "ResFood" диаметр...   546.0      1453.0   

                     category  \
0                Угорь речной 

In [146]:
products_df.to_excel('GFC_products.xlsx')

In [147]:
print(products_df.head())

                                                name   price  product_id  \
0  Угорь жареный мороженный 50% соуса 9/10/11  oz...  1275.5     11992.0   
1       Угорь жареный замороженный технический 11 OZ  1836.5     12106.0   
2  Филе желтоперого тунца Loin с/м  2-5кг, 1кг Re...   969.0      6950.0   
3    Филе желтоперого тунца САКУ с/м  500-800гр, 1кг  1173.5     12135.0   
4  Филе желтоперого тунца САКУ с/м  300-500гр, 1к...  1479.0       756.0   

       category                                       product_page  
0  Угорь речной  https://gfc-russia.ru/catalog/ryba-i-moreprodu...  
1  Угорь речной  https://gfc-russia.ru/catalog/ryba-i-moreprodu...  
2         Тунец  https://gfc-russia.ru/catalog/ryba-i-moreprodu...  
3         Тунец  https://gfc-russia.ru/catalog/ryba-i-moreprodu...  
4         Тунец  https://gfc-russia.ru/catalog/ryba-i-moreprodu...  
